In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler 

# Import dataset
X_train = pd.read_csv("C:/Users/steve/Desktop/Notebooks/Thesis-Project/Datasets/processed/X_train_processed.csv")
X_test = pd.read_csv("C:/Users/steve/Desktop/Notebooks/Thesis-Project/Datasets/processed/X_test_processed.csv")
y_train = pd.read_csv("C:/Users/steve/Desktop/Notebooks/Thesis-Project/Datasets/processed/y_train_processed.csv")
y_test = pd.read_csv("C:/Users/steve/Desktop/Notebooks/Thesis-Project/Datasets/processed/y_test_processed.csv")

In [42]:
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [15]:
# Group one
group_one = y_train[y_train["Diagnosis"] == 0]
group_two = y_train[y_train["Diagnosis"] == 3]

combined_groups_one = pd.concat([group_one, group_two], ignore_index = True)
combined_groups_one["Diagnosis"].unique()

array([0, 3], dtype=int64)

Is our target imbalanced for group one?

In [16]:
# Check percentage of each class
combined_groups_one["Diagnosis"].value_counts(normalize=True) * 100

Diagnosis
0    58.40708
3    41.59292
Name: proportion, dtype: float64

Not as much!

In [17]:
X_train.shape, X_test.shape

((425, 16), (183, 16))

In [ ]:
sc = StandardScaler()

# let's transform the data 
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape

((425, 16), (183, 16))

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

lg = LogisticRegression(multi_class = "auto", solver = "lbfgs", max_iter = 1000, random_state = 42)

svm = SVC(kernel ='rbf', decision_function_shape ='ovr', probability = True, random_state = 42)

dt = decision_tree_model = DecisionTreeClassifier(criterion ='gini', max_depth = 5, min_samples_split = 10,
                                                  min_samples_leaf = 5, max_features = 'sqrt', random_state = 42)

rf = RandomForestClassifier(n_estimators = 100, criterion = 'gini', max_depth = 5, min_samples_split = 10, 
                            min_samples_leaf = 5, max_features = 'sqrt', bootstrap = True, random_state = 42)

In [31]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import KFold, cross_validate

kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

# Define metrics to evaluate
scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average = 'weighted', zero_division=0),
    'recall': make_scorer(recall_score, average = 'weighted', zero_division=0),
    'f1': make_scorer(f1_score, average = 'weighted', zero_division=0),
    'roc_auc': make_scorer(roc_auc_score)
}

models  = {"Logistic Regression": lg, 
           "Support Vector Machine": svm, 
           "Decision Tree": dt, 
           "Random Forest": rf
}

model_data_mapping = {
    'Logistic Regression': X_train_scaled,
    'Support Vector Machine': X_train_scaled,
    'Decision Tree': X_train,
    'Random Forest': X_train
}

In [32]:
for model_name, model in models.items():

    X_train_to_use = model_data_mapping[model_name]
    
    results = cross_validate(model, 
                             X_train_to_use, 
                             y_train, 
                             scoring = scoring_metrics,
                             return_train_score = True,
                             cv = kf)
    print("------------------------------------------------------")
    print(model_name)
    for metric in scoring_metrics.keys():
            print(f'Mean train {metric}:', np.mean(results[f'train_{metric}']), '±', np.std(results[f'train_{metric}']))
            print(f'Mean test {metric}:', np.mean(results[f'test_{metric}']), '±', np.std(results[f'test_{metric}']))

c:\Users\steve\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\steve\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py"

------------------------------------------------------
Logistic Regression
Mean train accuracy: 0.6941176470588235 ± 0.01146681687624584
Mean test accuracy: 0.64 ± 0.03121529214452141
Mean train precision: 0.6871284262244898 ± 0.012349662904269539
Mean test precision: 0.6531650575161011 ± 0.018219520864726575
Mean train recall: 0.6941176470588235 ± 0.01146681687624584
Mean test recall: 0.64 ± 0.03121529214452141
Mean train f1: 0.6875366459443062 ± 0.011608383632557891
Mean test f1: 0.6393745655926393 ± 0.027455935848831044
Mean train roc_auc: nan ± nan
Mean test roc_auc: nan ± nan


c:\Users\steve\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py", line 633, in roc_auc_score
    raise ValueError("multi_class must be in ('ovo', 'ovr')")
ValueEr

------------------------------------------------------
Support Vector Machine
Mean train accuracy: 0.7752941176470587 ± 0.009593827311941234
Mean test accuracy: 0.583529411764706 ± 0.038375309247764895
Mean train precision: 0.7749055251185821 ± 0.010843378163222764
Mean test precision: 0.5800207410258946 ± 0.031212381300747563
Mean train recall: 0.7752941176470587 ± 0.009593827311941234
Mean test recall: 0.583529411764706 ± 0.038375309247764895
Mean train f1: 0.766803545357318 ± 0.009690197203232474
Mean test f1: 0.5691442896726835 ± 0.03606513035114108
Mean train roc_auc: nan ± nan
Mean test roc_auc: nan ± nan


c:\Users\steve\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py", line 633, in roc_auc_score
    raise ValueError("multi_class must be in ('ovo', 'ovr')")
ValueEr

------------------------------------------------------
Decision Tree
Mean train accuracy: 0.7005882352941176 ± 0.0158277929965573
Mean test accuracy: 0.4988235294117647 ± 0.05292810296095055
Mean train precision: 0.6959359884166603 ± 0.017541150550336504
Mean test precision: 0.5029860465849751 ± 0.07862160642862086
Mean train recall: 0.7005882352941176 ± 0.0158277929965573
Mean test recall: 0.4988235294117647 ± 0.05292810296095055
Mean train f1: 0.6811766023429656 ± 0.01960337050884977
Mean test f1: 0.4760410026044801 ± 0.05625859696676388
Mean train roc_auc: nan ± nan
Mean test roc_auc: nan ± nan


c:\Users\steve\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\steve\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.p

------------------------------------------------------
Random Forest
Mean train accuracy: 0.8070588235294117 ± 0.01551871289208578
Mean test accuracy: 0.6070588235294119 ± 0.03614656822526404
Mean train precision: 0.8224642737350333 ± 0.01632202609883332
Mean test precision: 0.599062218206253 ± 0.030309259206173195
Mean train recall: 0.8070588235294117 ± 0.01551871289208578
Mean test recall: 0.6070588235294119 ± 0.03614656822526404
Mean train f1: 0.795473180072406 ± 0.018022300017848804
Mean test f1: 0.581026339235055 ± 0.02435766322697153
Mean train roc_auc: nan ± nan
Mean test roc_auc: nan ± nan


c:\Users\steve\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\steve\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py", line 633, in roc_auc_score
    raise ValueError("multi_class must be in ('ovo', 'ovr')")
ValueEr